**a)**
I start by defining the Main object of this exercise, the `QuantumState` structure. It consists of an Vector (Array) of amplitudes as well the number of qubits. The i`th element represents the binary(n) state in the computational basis.
This definition enables the use of bitwise operations.

In [18]:
using LinearAlgebra, Printf

"""
Represents a quantum state in the computational basis
"""
struct QuantumState
    amplitudes::Vector{ComplexF64}
    n_qubits::Int
    #custom Constructor
    function QuantumState(amplitudes::Vector{ComplexF64})
        n = length(amplitudes)
        # Check if the length is a power of 2
        ispow2(n) ||throw(ArgumentError("Number of amplitudes must be a power of 2"))
        
        # Normalize the state vector
        norm_factor = norm(amplitudes)
        !(norm_factor ≈ 0) || throw(ArgumentError("State vector cannot be zero"))
        new(amplitudes / norm_factor, Int(log2(n)))
    end
end

"""
Creates a computational basis state |i⟩ for given integer i
"""
function basis_state(i::Integer, n_qubits::Integer)
    !(i >= 2^n_qubits) || throw(ArgumentError("Index must be less than 2^$n_qubits"))
    amplitudes = zeros(ComplexF64, 2^n_qubits)
    amplitudes[i + 1] = 1.0
    return QuantumState(amplitudes)
end



"""
Pretty print stuff
"""
function Base.show(io::IO, state::QuantumState)
    print(io, "Quantum state with $(state.n_qubits) qubits:\n")
    for (i, amplitude) in enumerate(state.amplitudes)
        if abs(amplitude) > 1e-10  # Only print non-zero amplitudes
            basis = string(i-1, base=2, pad=state.n_qubits)
            print(io, "  ($(round(amplitude, digits=3)))|$basis⟩\n")
        end
    end
end

Base.show

In [19]:
QuantumState(ComplexF64[1,1,1,1]) |>print

teststate = basis_state(0,2)

Quantum state with 2 qubits:
  (0.5 + 0.0im)|00⟩
  (0.5 + 0.0im)|01⟩
  (0.5 + 0.0im)|10⟩
  (0.5 + 0.0im)|11⟩


Quantum state with 2 qubits:
  (1.0 + 0.0im)|00⟩


**b).1** To implement the Not gate I use a bit mask, ``<<`` is the left shift operator. In the following "the bit" specifies the target_bit (0 based indexing!). Together with the `&` (bitwise `AND`) operator the mask will return `true` if the bit is not set. The `XOR` operation (`⊻`) is used to get the index of the state where the bit is set. In the last step the amplitudes of the state where the bit is set and non set are swaped.

In [20]:


"""
Applies a NOT gate to target_qubit
"""
function apply_not!(state::QuantumState, target_qubit::Integer)
     !(target_qubit >= state.n_qubits) || !(target_qubit < 0) || throw(ArgumentError("Invalid target qubit"))
    
    n = state.n_qubits
    mask = 1 << target_qubit  # Bit mask for target qubit
    
    # Swap amplitudes of traget_qubit
    for i in 0:(2^n-1)
        #Returns true if traget_qubit bit is not set
        if (i & mask) == 0
            # j is i with the target qubit flipped
            j = i ⊻ mask
            # Swap amplitudes
            state.amplitudes[i+1], state.amplitudes[j+1] = 
                state.amplitudes[j+1], state.amplitudes[i+1]
        end
    end
    
    return state
end

apply_not!

In [21]:
apply_not!(teststate,0)
teststate|>print

Quantum state with 2 qubits:
  (0.707 + 0.0im)|00⟩
  (0.707 + 0.0im)|11⟩


**b).2** For the Hadamard gate a bit mask is used (again). Instead of swapping the amplitudes, they are brought into a supperposition.

In [22]:
"""
Applies a Hadamard gate to target_qubit
"""
function apply_hadamard!(state::QuantumState, target_qubit::Integer)
    !(target_qubit >= state.n_qubits) || !(target_qubit < 0) || throw(ArgumentError("Invalid target qubit"))
    
    n = state.n_qubits
    mask = 1 << target_qubit  # Bit mask for target qubit
    factor = 1/sqrt(2)
    
    # Create temporary array to store new amplitudes
    new_amplitudes = similar(state.amplitudes)
    
    # bring amplitudes into supperposition, that differ only in target qubit
    for i in 0:(2^n-1)
        if (i & mask) == 0   #Returns true if traget_qubit bit is not set
            j = i ⊻ mask    # j is i with the target qubit flipped
            
            # Apply Hadamard transformation to the pair
            new_amplitudes[i+1] = factor * (state.amplitudes[i+1] + state.amplitudes[j+1]) # for |0⟩
            new_amplitudes[j+1] = factor * (state.amplitudes[i+1] - state.amplitudes[j+1]) # for |1⟩
        end
    end
    
    # Update state with new amplitudes
    state.amplitudes .= new_amplitudes
    return state
end

apply_hadamard!

In [23]:
apply_hadamard!(teststate,1)

Quantum state with 2 qubits:
  (0.707 + 0.0im)|01⟩
  (0.707 + 0.0im)|11⟩


**c)** The `CNOT` Gate is implemented using not one but two bit masks, one for the control and the other one for the target qubit. Instead to the single qubit gates, we only want an operation to happen when the control qubit is 1. If thats the case the the encoding of the qubits ensures that at index `i` one of the amplitudes to be flipped is located. The other index is found using the usual method using the target mask. Here the amplitudes of index `i` and `j` are not done directly swaped. Instead the swaping is archieved by iterating over the index `i` and `j` independetly.

In [24]:
function apply_cnot!(state::QuantumState, control_qubit::Integer, target_qubit::Integer)
    if control_qubit >= state.n_qubits || control_qubit < 0 ||
       target_qubit >= state.n_qubits || target_qubit < 0 ||
       control_qubit == target_qubit
        throw(ArgumentError("Invalid control or target qubit"))
    end
    
    n = state.n_qubits
    control_mask = 1 << control_qubit
    target_mask = 1 << target_qubit
    
    # Create temporary array to store new amplitudes
    new_amplitudes = copy(state.amplitudes)
    
    # Process all basis states
    for i in 0:(2^n-1)
        # Only apply operation if control qubit is 1
        if (i & control_mask) != 0
            # Flip the target qubit
            j = i ⊻ target_mask
            # Swap amplitudes
            new_amplitudes[i+1] = state.amplitudes[j+1]
        end
    end
    
    state.amplitudes .= new_amplitudes
    return state
end

apply_cnot! (generic function with 1 method)

In [25]:
state = basis_state(0,2)
state |> print
apply_hadamard!(state,0)
state |> print
apply_cnot!(state,0,1)
state |> print

Quantum state with 2 qubits:
  (1.0 + 0.0im)|00⟩
Quantum state with 2 qubits:
  (0.707 + 0.0im)|00⟩
  (0.707 + 0.0im)|01⟩
Quantum state with 2 qubits:
  (0.707 + 0.0im)|00⟩
  (0.707 + 0.0im)|11⟩
